# Imports

In [1]:
# cell 1 Imports
import os
import sys
import gc
import pickle
import sqlite3
from datetime import datetime
from pathlib import Path
from typing import Tuple, Dict, Any, List

import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import load_npz, csr_matrix, save_npz
from scipy.sparse.linalg import svds
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

try:
    from tqdm import tqdm
except Exception:
    tqdm = lambda x, **k: x

print("Imports complete. (Fast Mode + Retrain Ready)")

Imports complete. (Fast Mode + Retrain Ready)


# Path Definitions (Portable)

In [2]:
# cell 2 Path Definitions
PROJECT_ROOT = Path(os.getcwd())

# สร้าง Path structure
PROCESSED_PATH = PROJECT_ROOT / "processed"
CLEANED_PATH = PROCESSED_PATH / "cleaned"
PREPROCESS_PATH = PROCESSED_PATH / "preprocess"
MODEL_PATH = PROCESSED_PATH / "models"

# ไฟล์สำคัญสำหรับ Fast Mode & Feedback
DB_FILE = CLEANED_PATH / "ratings.db"
FEEDBACK_FILE = PROCESSED_PATH / "user_feedback.csv"
RATINGS_CSV = CLEANED_PATH / "ratings_cleaned_f.csv" # จำเป็นต้องใช้ตอน Retrain

# Utility function
def log(msg: str, level: str = "INFO") -> None:
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{level}] {ts} | {msg}")

log(f"Database File: {DB_FILE}")
log(f"Feedback File: {FEEDBACK_FILE}")

[INFO] 2025-11-18 06:45:05 | Database File: C:\Users\nonth\Documents\movie_predict_move\processed\cleaned\ratings.db
[INFO] 2025-11-18 06:45:05 | Feedback File: C:\Users\nonth\Documents\movie_predict_move\processed\user_feedback.csv


# Helper Functions

In [3]:
# cell 3 Load Data & Models (Fast Mode)

# Global Variables (Fast Mode)
movies_global = None
sim_sparse = None
U, Sigma, Vt, svd_user_mean = None, None, None, None
svd_user_index, svd_movie_index = {}, {}
svd_reverse_user_index, svd_reverse_movie_index = {}, {}

def load_svd_artifacts(model_dir: Path):
    log(f"Loading SVD artifacts from {model_dir}...")
    try:
        # โหลดแบบ Memory Map ถ้าทำได้ เพื่อประหยัด RAM (ถ้าไฟล์ใหญ่มาก)
        U = np.load(model_dir / "svd_U.npy")
        Sigma = np.load(model_dir / "svd_Sigma.npy")
        Vt = np.load(model_dir / "svd_Vt.npy")
        user_mean = np.load(model_dir / "svd_user_mean.npy")
        
        with open(model_dir / "svd_user_index.pkl", "rb") as f:
            user_index = pickle.load(f)
        with open(model_dir / "svd_movie_index.pkl", "rb") as f:
            movie_index = pickle.load(f)
        with open(model_dir / "svd_reverse_user_index.pkl", "rb") as f:
            reverse_user_index = pickle.load(f)
        with open(model_dir / "svd_reverse_movie_index.pkl", "rb") as f:
            reverse_movie_index = pickle.load(f)
            
        return {
            "U": U, "Sigma": Sigma, "Vt": Vt, "user_mean": user_mean,
            "user_index": user_index, "movie_index": movie_index,
            "reverse_user_index": reverse_user_index, "reverse_movie_index": reverse_movie_index
        }
    except Exception as e:
        log(f"SVD Artifacts not found: {e}", "WARN")
        return None

try:
    # 1. Movies (โหลดปกติ)
    movies_global = pd.read_csv(CLEANED_PATH / "movies_cleaned_f.csv")
    
    # 2. Ratings (ใช้ SQLite แทนการโหลดเข้า RAM)
    if not DB_FILE.exists():
        log(f"⚠️ WARNING: {DB_FILE} not found. Retrain might be needed to create it.", "WARN")
    else:
        log("✅ Database found. Using SQLite for ratings.")

    # 3. Similarity Matrix (โหลด CSR)
    sim_sparse = load_npz(MODEL_PATH / "content_similarity_sparse.npz")
    
    # 4. SVD Artifacts
    artifacts = load_svd_artifacts(MODEL_PATH)
    if artifacts:
        U = artifacts["U"]
        Sigma = artifacts["Sigma"]
        Vt = artifacts["Vt"]
        svd_user_mean = artifacts["user_mean"]
        svd_user_index = artifacts["user_index"]
        svd_movie_index = artifacts["movie_index"]
        svd_reverse_user_index = artifacts["reverse_user_index"]
        svd_reverse_movie_index = artifacts["reverse_movie_index"]
        log("--- MODELS LOADED SUCCESSFULLY ---")
    
except Exception as e:
    log(f"Error loading models: {e}", "ERROR")

[INFO] 2025-11-18 06:45:09 | ✅ Database found. Using SQLite for ratings.
[INFO] 2025-11-18 06:45:09 | Loading SVD artifacts from C:\Users\nonth\Documents\movie_predict_move\processed\models...
[INFO] 2025-11-18 06:45:10 | --- MODELS LOADED SUCCESSFULLY ---


# LOAD DATA & MODELS

In [4]:
# cell 4 Recommendation Logic (Fast Mode)

# Cache สำหรับ Hybrid
svd_preds_df_cache = pd.DataFrame()

def get_seen_movies_fast(user_id):
    """ดึงข้อมูลจาก SQLite"""
    try:
        with sqlite3.connect(DB_FILE) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT movieId FROM ratings WHERE userId = ?", (user_id,))
            return set([r[0] for r in cursor.fetchall()])
    except: return set()

def get_content_based_recs_fast(movie_title, top_n=10):
    movie_row = movies_global[movies_global['title'].str.contains(movie_title, case=False, na=False)]
    if movie_row.empty: return pd.DataFrame()
    movie_id = movie_row.iloc[0]['movieId']

    idx_arr = np.where(movies_global['movieId'].values == movie_id)[0]
    if idx_arr.size == 0: return pd.DataFrame()
    idx = int(idx_arr[0])

    try:
        row_vector = sim_sparse[idx]
        if hasattr(row_vector, 'indices'): 
            row_indices = row_vector.indices; row_data = row_vector.data
        else:
            row_vector = row_vector.tocoo(); row_indices = row_vector.col; row_data = row_vector.data

        similar_ids = [movies_global['movieId'].values[i] for i in row_indices]
        result = movies_global[movies_global.movieId.isin(similar_ids)][['movieId', 'title']].copy()
        score_map = dict(zip(similar_ids, row_data))
        result['similarity_score'] = result['movieId'].map(score_map)
        return result[result.movieId != movie_id].sort_values('similarity_score', ascending=False).head(top_n)
    except: return pd.DataFrame()

def get_cf_recs_for_user_fast(user_id, top_n=10):
    if user_id not in svd_user_index: return pd.DataFrame()
    u_idx = svd_user_index[user_id]
    user_vector = np.dot(U[u_idx, :], Sigma)
    preds = np.dot(user_vector, Vt) + svd_user_mean[u_idx]
    seen_movie_ids = get_seen_movies_fast(user_id)

    recs = []
    for i in range(len(preds)):
        if i in svd_reverse_movie_index:
            mid = svd_reverse_movie_index[i]
            if mid not in seen_movie_ids: recs.append((mid, preds[i]))

    recs.sort(key=lambda x: x[1], reverse=True)
    top_ids = [mid for mid, s in recs[:top_n]]
    result = movies_global[movies_global.movieId.isin(top_ids)][['movieId', 'title']].copy()
    score_map = dict(recs[:top_n])
    result['predicted_rating'] = result['movieId'].map(score_map)
    return result.sort_values('predicted_rating', ascending=False)

def get_cf_recs_for_movie_fast(movie_title, top_n=10):
    movie_row = movies_global[movies_global['title'].str.contains(movie_title, case=False, na=False)]
    if movie_row.empty: return pd.DataFrame()
    movie_id = movie_row.iloc[0]['movieId']

    if movie_id not in svd_movie_index: return pd.DataFrame()
    m_idx = svd_movie_index[movie_id]

    movie_factors = Vt[:, m_idx]
    user_scores = np.dot(np.dot(U, Sigma), movie_factors) + svd_user_mean
    top_indices = user_scores.argsort()[::-1][:top_n]
    
    results = []
    for u_idx in top_indices:
        if u_idx in svd_reverse_user_index:
            results.append({'userId': svd_reverse_user_index[u_idx], 'predicted_rating': user_scores[u_idx]})
    return pd.DataFrame(results)

def hybrid_score_fast(userId, movieId, alpha=0.7):
    try:
        svd_row = svd_preds_df_cache[svd_preds_df_cache.movieId == movieId]
        svd_score = float(svd_row.predicted_rating.values[0]) if not svd_row.empty else np.nan
    except: svd_score = np.nan

    idx_arr = np.where(movies_global['movieId'].values == movieId)[0]
    if idx_arr.size == 0: content_score = np.nan
    else:
        idx = int(idx_arr[0])
        try:
            row_vector = sim_sparse[idx]
            data = row_vector.data if hasattr(row_vector, 'data') else row_vector.toarray().flatten()
            content_score = float(np.nanmean(data[:50])) if len(data) > 0 else np.nan
        except: content_score = np.nan

    if np.isnan(svd_score) and np.isnan(content_score): return np.nan
    if np.isnan(svd_score): return content_score
    if np.isnan(content_score): return svd_score
    return alpha * svd_score + (1.0 - alpha) * content_score

def recommend_movies_fast(userId, top_n=10, alpha=0.7):
    global svd_preds_df_cache
    svd_preds_df_cache = get_cf_recs_for_user_fast(userId, top_n=500)
    candidates = list(svd_preds_df_cache['movieId'].values) if not svd_preds_df_cache.empty else []
    
    if len(candidates) < 50:
        seen = get_seen_movies_fast(userId)
        all_ids = movies_global['movieId'].values
        candidates.extend([mid for mid in all_ids if mid not in seen and mid not in candidates][:50])

    scores = []
    for mid in candidates:
        s = hybrid_score_fast(userId, mid, alpha)
        if not np.isnan(s): scores.append((mid, s))
            
    scores.sort(key=lambda x: x[1], reverse=True)
    top_ids = [mid for mid, s in scores[:top_n]]
    result = movies_global[movies_global.movieId.isin(top_ids)][['movieId', 'title']].copy()
    score_map = dict(scores[:top_n])
    result['hybrid_score'] = result['movieId'].map(score_map)
    return result.sort_values('hybrid_score', ascending=False).reset_index(drop=True)

log("Fast Mode Logic Defined.")

[INFO] 2025-11-18 06:45:12 | Fast Mode Logic Defined.


# 🚀 Interactive Test Dashboard (4-Tab Version)

In [5]:
# Cell 5: 🚀 Interactive Test Dashboard (Fast Mode + Enhanced Search)

log("Building Dashboard (with Enhanced Search UI)...")

# --- Helper: ฟังก์ชันค้นหาและแสดงผล (ใช้ร่วมกันเพื่อลดโค้ดซ้ำ) ---
def handle_movie_search(change, dropdown_widget, output_widget):
    with output_widget:
        clear_output()
        query = change['new']
        if len(query) < 3:
            dropdown_widget.options = []
            return
        
        # ค้นหาใน DataFrame
        results = movies_global[movies_global['title'].str.contains(query, case=False, na=False)]
        
        if results.empty:
            dropdown_widget.options = []
            print(f"❌ ไม่พบหนังที่ชื่อมีคำว่า: '{query}'")
        else:
            # อัปเดต Dropdown (Value = Title สำหรับโมเดลที่รับชื่อหนัง)
            # แสดง Title พร้อม Year (ถ้ามี) เพื่อให้เลือกง่ายขึ้น
            dropdown_widget.options = list(zip(results['title'], results['title']))
            
            # Show Preview Table
            print(f"✅ พบ {len(results)} เรื่อง (แสดง 5 เรื่องแรก):")
            display(results[['movieId', 'title']].head(5))

# --- Tab 1: Content-Based ---
ms_box1 = widgets.Text(placeholder='พิมพ์ชื่อหนัง (3 ตัวขึ้นไป)...', description='Search:')
ms_out1 = widgets.Output() # พื้นที่โชว์ผลค้นหา
ms_drop1 = widgets.Dropdown(options=[], description='Select:')
btn1 = widgets.Button(description='Test 1: Content-Based', button_style='info', icon='film')
out1 = widgets.Output()    # พื้นที่โชว์ผลลัพธ์โมเดล

# ผูก Event ค้นหา
ms_box1.observe(lambda c: handle_movie_search(c, ms_drop1, ms_out1), names='value')

def on_click1(b):
    with out1:
        clear_output()
        if not ms_drop1.value: print("⚠️ กรุณาเลือกหนังจาก Dropdown ก่อนครับ"); return
        display(Markdown(f"### 🎬 หนังที่เนื้อหาคล้าย: **{ms_drop1.value}**"))
        display(get_content_based_recs_fast(ms_drop1.value))
btn1.on_click(on_click1)

tab1 = widgets.VBox([ms_box1, ms_out1, ms_drop1, btn1, out1])

# --- Tab 2.1: User CF (User -> Movie) ---
uid_box21 = widgets.IntText(value=1, description='User ID:')
btn21 = widgets.Button(description='Test 2.1: User CF', button_style='primary', icon='user')
out21 = widgets.Output()

def on_click21(b):
    with out21:
        clear_output()
        display(Markdown(f"### 👤 แนะนำหนังให้ User ID: **{uid_box21.value}** (SVD)"))
        display(get_cf_recs_for_user_fast(uid_box21.value))
btn21.on_click(on_click21)

tab21 = widgets.VBox([uid_box21, btn21, out21])

# --- Tab 2.2: Movie CF (Movie -> Users) ---
ms_box22 = widgets.Text(placeholder='พิมพ์ชื่อหนัง (3 ตัวขึ้นไป)...', description='Search:')
ms_out22 = widgets.Output()
ms_drop22 = widgets.Dropdown(options=[], description='Select:')
btn22 = widgets.Button(description='Test 2.2: Movie CF', button_style='success', icon='users')
out22 = widgets.Output()

# ผูก Event ค้นหา
ms_box22.observe(lambda c: handle_movie_search(c, ms_drop22, ms_out22), names='value')

def on_click22(b):
    with out22:
        clear_output()
        if not ms_drop22.value: print("⚠️ กรุณาเลือกหนังจาก Dropdown ก่อนครับ"); return
        display(Markdown(f"### 👥 User ที่น่าจะชอบ: **{ms_drop22.value}**"))
        display(get_cf_recs_for_movie_fast(ms_drop22.value))
btn22.on_click(on_click22)

tab22 = widgets.VBox([ms_box22, ms_out22, ms_drop22, btn22, out22])

# --- Tab 3: Hybrid (Best) ---
uid_box3 = widgets.IntText(value=1, description='User ID:')
btn3 = widgets.Button(description='Test 3: Hybrid', button_style='danger', icon='star')
out3 = widgets.Output()

def on_click3(b):
    with out3:
        clear_output()
        display(Markdown(f"### 🤖 Hybrid Recommend for User ID: **{uid_box3.value}**"))
        display(recommend_movies_fast(uid_box3.value))
btn3.on_click(on_click3)

tab3 = widgets.VBox([uid_box3, btn3, out3])

# --- Display Tabs ---
tabs = widgets.Tab(children=[tab1, tab21, tab22, tab3])
tabs.set_title(0, '1. Content')
tabs.set_title(1, '2.1 User CF')
tabs.set_title(2, '2.2 Movie CF')
tabs.set_title(3, '3. Hybrid')
display(tabs)

[INFO] 2025-11-18 06:45:14 | Building Dashboard (with Enhanced Search UI)...


## ตัวอย่าง Interactive Test (เก่า)
<table>
  <tr>
    <td align="center">
      <img src="./images/test1.png" alt="ผลลัพธ์ Test 1: Content-Based" width="400">
      <br>
      <b>Test 1: Content-Based (Input: Movie)</b>
    </td>
    <td align="center">
      <img src="./images/test2.1.png" alt="ผลลัพธ์ Test 2.1: CF (User)" width="400">
      <br>
      <b>Test 2.1: Collaborative Filtering (Input: User)</b>
    </td>
  </tr>
  <tr>
    <td align="center">
      <<img src="./images/test2.2.png" alt="ผลลัพธ์ Test 2.2: CF (Movie)" width="400">
      <br>
      <b>Test 2.2: Collaborative Filtering (Input: Movie)</b>
    </td>
    <td align="center">
      <img src="./images/test3.png" alt="ผลลัพธ์ Test 3: Hybrid" width="400">
      <br>
      <b>Test 3: Hybrid Model (Input: User)</b>
    </td>
  </tr>
</table>

# ⭐️ Feedback Widget

In [6]:
# cell 6 User Feedback Loop (Enhanced Search UI)

# ตรวจสอบไฟล์ Feedback
if not FEEDBACK_FILE.exists():
    pd.DataFrame(columns=['userId', 'movieId', 'rating', 'timestamp']).to_csv(FEEDBACK_FILE, index=False)
    log("Created new feedback file.")

display(Markdown("### 📝 Give Feedback (เพิ่มข้อมูลใหม่)"))

# UI Components
fb_user_id = widgets.IntText(value=None, description='User ID:')
fb_movie_search = widgets.Text(placeholder='พิมพ์ชื่อหนัง 3 ตัวอักษร...', description='Search:')
fb_search_output = widgets.Output() # <-- เพิ่ม Output ตรงนี้สำหรับโชว์ผลค้นหา
fb_movie_dropdown = widgets.Dropdown(description='Select:')
fb_rating = widgets.FloatSlider(value=5.0, min=0.5, max=5.0, step=0.5, description='Rating:')
fb_submit_btn = widgets.Button(description='Submit Feedback', button_style='success', icon='save')
fb_status_output = widgets.Output()

# Logic ค้นหาแบบเต็มรูปแบบ
def on_fb_search_change(change):
    with fb_search_output:
        clear_output()
        search_term = change['new']
        if len(search_term) < 3:
            fb_movie_dropdown.options = []
            return
        
        matches = movies_global[movies_global['title'].str.contains(search_term, case=False, na=False)]
        
        if matches.empty:
            fb_movie_dropdown.options = []
            print(f"❌ ไม่พบหนัง: '{search_term}'")
        else:
            # สำหรับ Feedback เราต้องเก็บ MovieID (Value = movieId)
            fb_movie_dropdown.options = list(zip(matches['title'], matches['movieId']))
            print(f"✅ พบ {len(matches)} เรื่อง:")
            display(matches[['movieId', 'title']].head(5))

fb_movie_search.observe(on_fb_search_change, names='value')

def on_fb_submit(b):
    with fb_status_output:
        clear_output()
        if not fb_movie_dropdown.value:
            print("❌ กรุณาเลือกหนังก่อนกด Submit")
            return
        
        new_row = {
            'userId': fb_user_id.value,
            'movieId': fb_movie_dropdown.value, # ค่า Value คือ ID
            'rating': fb_rating.value,
            'timestamp': int(datetime.now().timestamp())
        }
        
        # Save to CSV
        pd.DataFrame([new_row]).to_csv(FEEDBACK_FILE, mode='a', header=False, index=False)
        
        # Show Success
        movie_name = movies_global[movies_global.movieId == new_row['movieId']].iloc[0]['title']
        print(f"✅ บันทึกสำเร็จ!")
        print(f"   User: {new_row['userId']}")
        print(f"   Movie: {movie_name} (ID: {new_row['movieId']})")
        print(f"   Rating: {new_row['rating']}")
        
        # Reset Search (Optional)
        fb_movie_search.value = ''
        fb_movie_dropdown.options = []
        fb_search_output.clear_output()

fb_submit_btn.on_click(on_fb_submit)

# Layout
display(widgets.VBox([
    fb_user_id, 
    fb_movie_search, 
    fb_search_output,     # โชว์ตารางผลค้นหา
    fb_movie_dropdown,    # เลือกหนัง
    fb_rating, 
    fb_submit_btn, 
    fb_status_output      # โชว์สถานะบันทึก
]))

### 📝 Give Feedback (เพิ่มข้อมูลใหม่)

## ตัวอย่าง Feedback Widget (เก่า)
<img src="./images/feedback_ui.png" alt="test2.1" width="40%">

## ตัวอย่างข้อมูล user_feedback.csv
<img src="./images/feedback_csv_example.png" alt="test2.1" width="60%">

# 🤖 Retrain Master Function

In [7]:
# cell 7 Retrain Model (Integrated with Fast Mode)

retrain_btn = widgets.Button(description='Retrain Model', button_style='danger', layout=widgets.Layout(width='100%'), icon='cogs')
retrain_output = widgets.Output()

def retrain_pipeline(b):
    with retrain_output:
        clear_output()
        log("🚀 Starting Retrain Pipeline...")
        
        try:
            # 1. Load Full Data (Temporary Load)
            log("Loading full dataset for training...")
            if RATINGS_CSV.exists():
                df_train = pd.read_csv(RATINGS_CSV)
            else:
                # Fallback: Read from DB if CSV missing
                conn = sqlite3.connect(DB_FILE)
                df_train = pd.read_sql("SELECT * FROM ratings", conn)
                conn.close()

            # 2. Merge Feedback
            if FEEDBACK_FILE.exists():
                df_fb = pd.read_csv(FEEDBACK_FILE)
                if not df_fb.empty:
                    log(f"Merging {len(df_fb)} feedback rows...")
                    df_train = pd.concat([df_train, df_fb], ignore_index=True)
            
            # 3. Retrain SVD
            log("Computing SVD...")
            # Create Sparse Matrix
            user_ids = df_train['userId'].unique()
            movie_ids = df_train['movieId'].unique()
            
            # Re-map indices
            u_map = {uid: i for i, uid in enumerate(user_ids)}
            m_map = {mid: i for i, mid in enumerate(movie_ids)}
            
            row = df_train['userId'].map(u_map).values
            col = df_train['movieId'].map(m_map).values
            data = df_train['rating'].values
            
            R_sparse = csr_matrix((data, (row, col)), shape=(len(user_ids), len(movie_ids)))
            
            # Compute SVD
            U_new, Sigma_new, Vt_new = svds(R_sparse, k=50)
            Sigma_new = np.diag(Sigma_new)
            
            # User Mean
            user_mean_new = df_train.groupby('userId')['rating'].mean().reindex(user_ids).fillna(0).values
            
            # Reverse Maps
            rev_u_map = {v: k for k, v in u_map.items()}
            rev_m_map = {v: k for k, v in m_map.items()}

            # 4. Save Artifacts
            log("Saving new model artifacts...")
            np.save(MODEL_PATH / "svd_U.npy", U_new)
            np.save(MODEL_PATH / "svd_Sigma.npy", Sigma_new)
            np.save(MODEL_PATH / "svd_Vt.npy", Vt_new)
            np.save(MODEL_PATH / "svd_user_mean.npy", user_mean_new)
            
            with open(MODEL_PATH / "svd_user_index.pkl", "wb") as f: pickle.dump(u_map, f)
            with open(MODEL_PATH / "svd_movie_index.pkl", "wb") as f: pickle.dump(m_map, f)
            with open(MODEL_PATH / "svd_reverse_user_index.pkl", "wb") as f: pickle.dump(rev_u_map, f)
            with open(MODEL_PATH / "svd_reverse_movie_index.pkl", "wb") as f: pickle.dump(rev_m_map, f)
            
            # 5. Important: Update Global Variables (Hot Reload)
            global U, Sigma, Vt, svd_user_mean, svd_user_index, svd_movie_index, svd_reverse_user_index, svd_reverse_movie_index
            U, Sigma, Vt = U_new, Sigma_new, Vt_new
            svd_user_mean = user_mean_new
            svd_user_index, svd_movie_index = u_map, m_map
            svd_reverse_user_index, svd_reverse_movie_index = rev_u_map, rev_m_map

            # 6. Important: Update SQLite DB (For Fast Mode)
            log("Updating Database for Fast Mode...")
            # เพื่อความง่าย เราจะ overwrite ตาราง ratings ใหม่
            conn = sqlite3.connect(DB_FILE)
            # ลบข้อมูลเก่าเฉพาะส่วน feedback หรือจะลงทับหมดก็ได้ (ลงทับชัวร์สุด)
            df_train[['userId', 'movieId']].to_sql('ratings', conn, if_exists='replace', index=False)
            conn.execute("CREATE INDEX IF NOT EXISTS idx_user ON ratings(userId)")
            conn.close()

            # 7. Cleanup RAM
            del df_train, R_sparse, U_new, Sigma_new, Vt_new
            gc.collect()
            
            log("✅ Retrain Complete! Dashboard updated successfully.")
            
        except Exception as e:
            log(f"Retrain Failed: {e}", "ERROR")

retrain_btn.on_click(retrain_pipeline)
display(retrain_btn, retrain_output)

Button(button_style='danger', description='Retrain Model', icon='cogs', layout=Layout(width='100%'), style=But…

Output()

## ตัวอย่างการแสดงผลเมื่อ Retrain สำเร็จ(เก่า)
<img src="./images/retrain_modelling_example.png" alt="retrain_modelling" width="80%">